# Partition-Aware Merge Materialization

Perform merge (upsert) operations **scoped to affected partitions only**, dramatically reducing I/O.

### Before vs After

| Metric | Full-Table Merge | Partition-Aware Merge |
|---|---|---|
| Data read | Entire table | Affected partitions only |
| Data written | Entire table | Affected partitions only |
| Concurrent writes | ❌ Unsafe | ✅ Safe per-partition |
| Memory usage | Table size | Partition size |

## Setup

In [ ]:
from pathlib import Path
import shutil
import pandas as pd

from lakelogic.core.models import (
    DataContract, Materialization, Model, FieldDefinition, Info, Server
)
from lakelogic.core.materialization import materialize_dataframe

BASE   = Path(".")
OUTPUT = BASE / "data" / "silver" / "orders"

# Clean previous runs
if OUTPUT.exists():
    shutil.rmtree(OUTPUT)

# Build contract
contract = DataContract(
    version="1.0",
    primary_key=["order_id"],
    info=Info(title="Silver Orders", version="1.0"),
    model=Model(fields=[
        FieldDefinition(name="order_id", type="string", required=True),
        FieldDefinition(name="customer_id", type="string", required=True),
        FieldDefinition(name="region", type="string", required=True),
        FieldDefinition(name="order_date", type="date"),
        FieldDefinition(name="amount", type="float"),
        FieldDefinition(name="status", type="string"),
    ]),
    materialization=Materialization(
        strategy="merge",
        partition_by=["region"],
        target_path=str(OUTPUT),
        format="parquet",
    ),
    server=Server(type="file", path=str(OUTPUT), format="parquet"),
)

print(f"Contract: strategy={contract.materialization.strategy}, partition_by={contract.materialization.partition_by}")

## Batch 1: Initial Load

6 orders across **3 regions** (US, EU, APAC).

In [ ]:
batch1 = pd.read_csv("data/batch_1.csv")
batch1

In [ ]:
result1 = materialize_dataframe(batch1, contract)
print(f"Written: {result1['rows_written']} rows")

# Show partition structure
print("\nPartition directories:")
for d in sorted(OUTPUT.iterdir()):
    if d.is_dir():
        files = list(d.glob("*.parquet"))
        rows = pd.read_parquet(files[0]) if files else pd.DataFrame()
        print(f"  {d.name}/  ({len(rows)} rows)")

## Batch 2: US-Only Update

This batch:
- **Updates** ORD-001: pending → shipped
- **Updates** ORD-004: pending → shipped  
- **Inserts** ORD-007: new US order

**EU and APAC partitions should NOT be read or written.**

In [ ]:
batch2 = pd.read_csv("data/batch_2.csv")
batch2

In [ ]:
import time

# Record file timestamps BEFORE the merge
eu_mtime = (OUTPUT / "region=EU" / "data.parquet").stat().st_mtime
apac_mtime = (OUTPUT / "region=APAC" / "data.parquet").stat().st_mtime
time.sleep(0.1)  # Ensure timestamp resolution

# Run partition-aware merge
result2 = materialize_dataframe(batch2, contract)
print(f"Written: {result2['rows_written']} rows")

## Verify: US Partition Updated

In [ ]:
us_data = pd.read_parquet(OUTPUT / "region=US" / "data.parquet")
print(f"US partition: {len(us_data)} rows")
us_data

In [ ]:
# Verify the updates
ord001 = us_data[us_data["order_id"] == "ORD-001"].iloc[0]
print(f"✅ ORD-001 status: {ord001['status']}  (was: pending)")

ord007 = us_data[us_data["order_id"] == "ORD-007"]
print(f"✅ ORD-007 inserted: ${ord007.iloc[0]['amount']:.2f}")

## Verify: EU & APAC Partitions UNTOUCHED

In [ ]:
eu_data = pd.read_parquet(OUTPUT / "region=EU" / "data.parquet")
eu_unchanged = (OUTPUT / "region=EU" / "data.parquet").stat().st_mtime == eu_mtime
print(f"EU partition: {len(eu_data)} rows | File unchanged: {eu_unchanged}")
eu_data

In [ ]:
apac_data = pd.read_parquet(OUTPUT / "region=APAC" / "data.parquet")
apac_unchanged = (OUTPUT / "region=APAC" / "data.parquet").stat().st_mtime == apac_mtime
print(f"APAC partition: {len(apac_data)} rows | File unchanged: {apac_unchanged}")
apac_data

## Summary

The second batch:
- **Touched**: 1 partition (US) — 2 updates + 1 insert
- **Skipped**: 2 partitions (EU, APAC) — completely untouched
- **I/O saved**: ~67% (only 1 of 3 partitions was read/written)

For a table with 365 daily partitions where today's batch only touches 1 day, the savings would be **~99.7%**.

In [ ]:
print("Summary:")
print(f"  Batch 2 touched:  1 partition (US)")
print(f"  Batch 2 skipped:  2 partitions (EU, APAC)")
print(f"  US rows merged:   {len(us_data)} (2 updated + 1 new)")
print(f"  Total I/O saved:  ~67%")